In [14]:
import tensorflow as tf
import os
import numpy as np

# show the tensorflow package version
print(tf.__version__)

2.13.0-rc0


Load the saved models from `../02_model_training/saved_models/`. 

In [24]:
export_dir = '/Users/kgoldmann/Documents/Projects/AMBER/on_device_classifier/02_model_training/saved_models/1/'

# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

2023-05-22 16:02:51.015255: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-22 16:02:51.015267: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-22 16:02:51.015412: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /Users/kgoldmann/Documents/Projects/AMBER/on_device_classifier/02_model_training/saved_models/1/
2023-05-22 16:02:51.016349: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-05-22 16:02:51.016356: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /Users/kgoldmann/Documents/Projects/AMBER/on_device_classifier/02_model_training/saved_models/1/
2023-05-22 16:02:51.018661: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-05-22 16:02:51.087973: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at pat

In [31]:
import pathlib
import re
tflite_model_file = re.sub("02_model_training", "03_model_compression", export_dir)
tflite_model_file = pathlib.Path(tflite_model_file + 'model.tflite')

tflite_model_file.write_bytes(tflite_model)

20154092

In [33]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_rescaling_1_input:0', 'index': 0, 'shape': array([  1, 300, 300,   3], dtype=int32), 'shape_signature': array([ -1, 300, 300,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 20, 'shape': array([1, 3], dtype=int32), 'shape_signature': array([-1,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [34]:
to_predict = np.array([[1.0]], dtype=np.float32)
print(to_predict)

interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[1.]]
[[-1.2376655   0.47881597  0.78507096]]
